In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import LongType,StringType, IntegerType, StructField, StructType, DoubleType
from delta import *

In [2]:
# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
spark.sql(
    """
    SHOW DATABASES
    """
).show()

+------------+
|   namespace|
+------------+
|     default|
|        demo|
|pl7g1_silver|
+------------+



In [4]:
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_SILVER.Incendios
    """
)

DataFrame[]

In [5]:
spark.sql(
    """
    DROP DATABASE PL7G1_SILVER CASCADE
    """
)

DataFrame[]

In [6]:
spark.sql(
    """
    CREATE DATABASE PL7G1_SILVER LOCATION 'hdfs://hdfs-nn:9000//Poluicao_ar/Silver/PL7G1_SILVER.db/'
    """
)

DataFrame[]

In [7]:
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_SILVER.Incendios
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_SILVER.Incendios (
        date DATE,
        time VARCHAR(10),
        am_pm VARCHAR(2),
        battalion VARCHAR(15),
        community_District VARCHAR(15),
        neighborhoods VARCHAR(400),
        precinct VARCHAR(15),
        incident_Classification VARCHAR(31),
        cause_Fire_Description VARCHAR(51),
        fire_Code_Category VARCHAR(21)
)
    USING DELTA
    PARTITIONED BY (
        case_Year INT,
        borough  VARCHAR(15)
    )
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/Incendios/'
    """
)


DataFrame[]

In [8]:
spark.sql(
    """
    SHOW TABLES FROM PL7G1_SILVER
    """
).show()

+------------+---------+-----------+
|   namespace|tableName|isTemporary|
+------------+---------+-----------+
|pl7g1_silver|incendios|      false|
+------------+---------+-----------+



In [9]:
spark.sql(
    """
    SELECT *
    FROM PL7G1_SILVER.Incendios
    """
).show()

+----+----+-----+---------+------------------+-------------+--------+-----------------------+----------------------+------------------+---------+-------+
|date|time|am_pm|battalion|community_District|neighborhoods|precinct|incident_Classification|cause_Fire_Description|fire_Code_Category|case_Year|borough|
+----+----+-----+---------+------------------+-------------+--------+-----------------------+----------------------+------------------+---------+-------+
+----+----+-----+---------+------------------+-------------+--------+-----------------------+----------------------+------------------+---------+-------+



In [10]:
spark.sql(
    """
    DESCRIBE FORMATTED PL7G1_SILVER.Incendios
    """
).toPandas()

col_name  \
0                           date   
1                           time   
2                          am_pm   
3                      battalion   
4             community_District   
5                  neighborhoods   
6                       precinct   
7        incident_Classification   
8         cause_Fire_Description   
9             fire_Code_Category   
10                     case_Year   
11                       borough   
12                                 
13                # Partitioning   
14                        Part 0   
15                        Part 1   
16                                 
17  # Detailed Table Information   
18                          Name   
19                      Location   
20                      Provider   
21                         Owner   
22                      External   
23              Table Properties   

                                            data_type comment  
0                                                date          
1                                              string          
2                                              string          
3                                              string          
4                                              string          
5                                              string          
6                                              string          
7                                              string          
8                                              string          
9                                              string          
10                                                int          
11                                             string          
12                                                             
13                                                             
14                                          case_Year          
15                                            borough          
16                                                             
17                                                             
18                             pl7g1_silver.incendios          
19  hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_S...          
20                                              delta          
21                                             jovyan          
22                                               true          
23  [delta.minReaderVersion=1,delta.minWriterVersi...

In [22]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Bureau_of_Fire_Investigations_-_Fire_Causes.csv"

customSchema = StructType([
    StructField("case_Year", IntegerType(), True),
    StructField("Incident_DateTime", StringType(), True),  
    StructField("borough", StringType(), True),
    StructField("battalion", StringType(), True),
    StructField("community_District", StringType(), True),
    StructField("precinct", StringType(), True),
    StructField("incident_Classification", StringType(), True),
    StructField("cause_Fire_Description", StringType(), True),
    StructField("fire_Code_Category", StringType(), True)
])

incendios = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

incendios.show(30)
#sales_df.printSchema()

+---------+--------------------+-------------+---------+------------------+--------+-----------------------+----------------------+--------------------+
|case_Year|   Incident_DateTime|      borough|battalion|community_District|precinct|incident_Classification|cause_Fire_Description|  fire_Code_Category|
+---------+--------------------+-------------+---------+------------------+--------+-----------------------+----------------------+--------------------+
|     2016|01/01/2016 12:08:...|       Queens|       50|                08|     107|   MD Residential St...|  Smoking (Cigarett...|             Smoking|
|     2016|01/01/2016 12:13:...|Staten Island|       21|                02|     122|   PD Residential St...|  Incendiary - Comb...|          Incendiary|
|     2016|01/01/2016 12:21:...|       Queens|       37|                05|     104|   MD Residential St...|  Incendiary - Comb...|          Incendiary|
|     2016|01/01/2016 12:29:...|        Bronx|       15|                12|     04

In [23]:
#substituir os valores em branco por "Sem Informação"

from pyspark.sql.functions import when, col, concat, lit

incendios = incendios.withColumn(
    "borough",
    when(
        (col("borough").isNull() | (col("borough") == None)), 
        "Sem Informação"
    ).otherwise(col("borough")))


incendios = incendios.withColumn(
    "battalion",
    when(
        (col("battalion").isNull() | (col("battalion") == None)), 
        "Sem Informação"
    ).otherwise(col("battalion")))


incendios = incendios.withColumn(
    "community_District",
    when(
        (col("community_District").isNull() | (col("community_District") == None)), 
        "Sem Informação"
    ).otherwise(col("community_District")))

incendios = incendios.withColumn(
    "precinct",
    when(
        (col("precinct").isNull() | (col("precinct") == None)), 
        "Sem Informação"
    ).otherwise(col("precinct")))

incendios = incendios.withColumn(
    "incident_Classification",
    when(
        (col("incident_Classification").isNull() | (col("incident_Classification") == None)), 
        "Sem Informação"
    ).otherwise(col("incident_Classification")))

incendios = incendios.withColumn(
    "cause_Fire_Description",
    when(
        (col("cause_Fire_Description").isNull() | (col("cause_Fire_Description") == None)), 
        "Sem Informação"
    ).otherwise(col("cause_Fire_Description")))


incendios = incendios.withColumn(
    "fire_Code_Category",
    when(
        (col("fire_Code_Category").isNull() | (col("fire_Code_Category") == None)), 
        "Sem Informação"
    ).otherwise(col("fire_Code_Category")))


In [24]:
#separar o IncidentDateTime em DateTime e AM/PM
from pyspark.sql.functions import when, col, concat, lit, split

incendios= incendios.withColumn("date",split(col("Incident_DateTime")," ").getItem(0)) \
.withColumn("time",split(col("Incident_DateTime")," ").getItem(1)) \
.withColumn("am_pm",split(col("Incident_DateTime")," ").getItem(2)) \
.drop("Incident_DateTime")

incendios.printSchema()
incendios.show()


root
 |-- case_Year: integer (nullable = true)
 |-- borough: string (nullable = true)
 |-- battalion: string (nullable = true)
 |-- community_District: string (nullable = true)
 |-- precinct: string (nullable = true)
 |-- incident_Classification: string (nullable = true)
 |-- cause_Fire_Description: string (nullable = true)
 |-- fire_Code_Category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- am_pm: string (nullable = true)

+---------+-------------+---------+------------------+--------+-----------------------+----------------------+--------------------+----------+--------+-----+
|case_Year|      borough|battalion|community_District|precinct|incident_Classification|cause_Fire_Description|  fire_Code_Category|      date|    time|am_pm|
+---------+-------------+---------+------------------+--------+-----------------------+----------------------+--------------------+----------+--------+-----+
|     2016|       Queens|       50|     

In [25]:
#substituit o / no DateTime para -
from pyspark.sql.functions import regexp_replace
incendios=incendios.withColumn('date', regexp_replace('date', '/', '-')) 
incendios.show(truncate=False)

+---------+-------------+---------+------------------+--------+--------------------------+-------------------------------------------------+--------------------+----------+--------+-----+
|case_Year|borough      |battalion|community_District|precinct|incident_Classification   |cause_Fire_Description                           |fire_Code_Category  |date      |time    |am_pm|
+---------+-------------+---------+------------------+--------+--------------------------+-------------------------------------------------+--------------------+----------+--------+-----+
|2016     |Queens       |50       |08                |107     |MD Residential Structure  |Smoking (Cigarette/Cigar)                        |Smoking             |01-01-2016|12:08:00|AM   |
|2016     |Staten Island|21       |02                |122     |PD Residential Structure  |Incendiary - Combustible Material                |Incendiary          |01-01-2016|12:13:00|AM   |
|2016     |Queens       |37       |05                |104   

In [26]:
#cast de string para date (a data)
from pyspark.sql.functions import *
incendios=incendios.select(to_date(incendios.date, 'MM-dd-yyy').alias("date"),"time","am_pm","case_Year","borough","battalion", "community_District", "precinct","incident_Classification", "cause_Fire_Description","fire_Code_Category") 


In [27]:
incendios.select("precinct").distinct().show(200)

+--------------+
|      precinct|
+--------------+
|           030|
|           088|
|           009|
|           101|
|           032|
|           073|
|            69|
|           112|
|            NJ|
|           113|
|           048|
|           028|
|           052|
|           061|
|            NA|
|           069|
|           060|
|           013|
|           107|
|           110|
|           024|
|           067|
|           075|
|           081|
|Sem Informação|
|           034|
|           006|
|           100|
|           019|
|            75|
|           046|
|           120|
|           020|
|            46|
|           033|
|           047|
|           071|
|            77|
|           040|
|           025|
|           104|
|            60|
|           076|
|           066|
|            41|
|           102|
|           042|
|           111|
|           044|
|          PSA2|
|           5th|
|           103|
|           045|
|            44|
|           115|
|          NC0

In [28]:
#vamos remover os "0" que estão nas colunas battalion e prectint. Fazemos estas alterações para conseguirmos dar join posteriormente com os datasets complementares.
from pyspark.sql.functions import * 
import pyspark.sql.functions as F

incendios = incendios.withColumn("battalion", F.regexp_replace("battalion", r'^[0]*',''))
incendios = incendios.withColumn("precinct", F.regexp_replace("precinct", r'^[0]*',''))

incendios= incendios.withColumn("precinct", 
   when((incendios.precinct == "5th"), lit("5"))
   .when((incendios.precinct == "7th"), lit("7")) 
   .otherwise(col("precinct"))) 

                            


In [29]:
incendios.select("precinct").distinct().show(200)

+--------------+
|      precinct|
+--------------+
|             7|
|           101|
|            69|
|           112|
|            42|
|            73|
|            NJ|
|           113|
|            30|
|            34|
|            28|
|            22|
|            52|
|            NA|
|            71|
|            47|
|           107|
|           110|
|            43|
|Sem Informação|
|             5|
|           100|
|            18|
|            70|
|            61|
|            75|
|            17|
|            26|
|           120|
|            46|
|            78|
|            77|
|             6|
|           104|
|            60|
|            68|
|            90|
|            19|
|            41|
|            23|
|           102|
|           111|
|          PSA2|
|           103|
|            40|
|            25|
|            44|
|           115|
|          NC08|
|           122|
|           108|
|           mtn|
|          NC05|
|            81|
|           114|
|            3

In [30]:
incendios.filter(incendios.borough=="Manhattan").select("community_District").distinct().show() 

+------------------+
|community_District|
+------------------+
|                07|
|                15|
|                11|
|        Out of NYC|
|                01|
|    Sem Informação|
|                17|
|                09|
|                05|
|                08|
|                03|
|                02|
|                06|
|                10|
|                12|
|                04|
+------------------+



In [31]:
incendios = incendios.withColumn("Neighborhoods", lit("Sem Informação"))
incendios.show()

+----------+--------+-----+---------+-------------+---------+------------------+--------+-----------------------+----------------------+--------------------+--------------+
|      date|    time|am_pm|case_Year|      borough|battalion|community_District|precinct|incident_Classification|cause_Fire_Description|  fire_Code_Category| Neighborhoods|
+----------+--------+-----+---------+-------------+---------+------------------+--------+-----------------------+----------------------+--------------------+--------------+
|2016-01-01|12:08:00|   AM|     2016|       Queens|       50|                08|     107|   MD Residential St...|  Smoking (Cigarett...|             Smoking|Sem Informação|
|2016-01-01|12:13:00|   AM|     2016|Staten Island|       21|                02|     122|   PD Residential St...|  Incendiary - Comb...|          Incendiary|Sem Informação|
|2016-01-01|12:21:00|   AM|     2016|       Queens|       37|                05|     104|   MD Residential St...|  Incendiary - Comb...

In [ ]:
#cada community District tem vários bairros, vamos relacionar o numero com os bairros
from pyspark.sql.functions import when, col, concat, lit, split

incendios= incendios.withColumn("Neighborhoods", 
   when((incendios.borough == "Bronx") & (incendios.community_District == "01"), lit("Mott Haven, Port Morris, Melrose") ) 
     .when((incendios.borough == "Bronx") & (incendios.community_District == "02"), lit("Hunts Point, Longwood, Morrisania")) 
     .when((incendios.borough == "Bronx") & (incendios.community_District == "03"), lit("Crotona Park, Claremont Village, Concourse Village, Woodstock, and Morrisania")) 
     .when((incendios.borough == "Bronx") & (incendios.community_District == "04"), lit("Highbridge, Concourse, Mount Eden, and Concourse Village")) 
     .when((incendios.borough == "Bronx") & (incendios.community_District == "05"), lit("Fordham, University Heights, Morris Heights, Bathgate, and Mount Hope")) 
     .when((incendios.borough == "Bronx") & (incendios.community_District == "06"), lit("Belmont, Bathgate, West Farms, East Tremont, and Bronx Park South")) 
     .when((incendios.borough == "Bronx") & (incendios.community_District == "07"), lit("Norwood, University Heights, Jerome Park, Bedford Park, Fordham, and Kingsbridge Heights")) 
     .when((incendios.borough == "Bronx") & (incendios.community_District == "08"), lit("Fieldston, Kingsbridge, Kingsbridge Heights, Marble Hill, Riverdale, Spuyten Duyvil, Van Cortlandt Village")) 
     .when((incendios.borough == "Bronx") & (incendios.community_District == "09"), lit("Parkchester, Unionport, Soundview, Castle Hill, Bruckner, Harding Park, Bronx River and Clason Point")) 
     .when((incendios.borough == "Bronx") & (incendios.community_District == "10"), lit("Co-op City, City Island, Spencer Estates, Throggs Neck, Country Club, Zerega, Westchester Square, Pelham Bay, Eastchester Bay, Schuylerville, Edgewater, Locust Point, and Silver Beach"))   
     .when((incendios.borough == "Bronx") & (incendios.community_District == "11"), lit("Allerton, Bronx Park East, Eastchester Gardens, Indian Village, Morris Park, Parkside, Pelham Gardens, Pelham Parkway, and Van Nest"))  
     .when((incendios.borough == "Bronx") & (incendios.community_District == "12"), lit("Edenwald, Wakefield, Williamsbridge, Woodlawn, Fish Bay, Eastchester, Olinville, and Baychester"))   
    
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "01"), lit("Flushing Avenue, Williamsburg, Greenpoint, Northside, and Southside"))   
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "02"), lit("Brooklyn Heights, Fulton Mall, Boerum Hill, Fort Greene, Brooklyn Navy Yard, Fulton Ferry, and Clinton Hill")) 
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "03"), lit("Bedford-Stuyvesant, Stuyvesant Heights, and Ocean Hill")) 
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "04"), lit("Bushwick")) 
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "05"), lit("East New York, Cypress Hills, Highland Park, New Lots, City Line, Starrett City, and Ridgewood")) 
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "06"), lit("Red Hook, Carroll Gardens, Park Slope, Gowanus, and Cobble Hill")) 
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "07"), lit("Sunset Park and Windsor Terrace")) 
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "08"), lit("Crown Heights, Prospect Heights, and Weeksville")) 
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "09"), lit("Crown Heights, Prospect Lefferts Gardens, and Wingate")) 
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "10"), lit("Bay Ridge, Dyker Heights, and Fort Hamilton"))   
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "11"), lit("Bath Beach, Gravesend, Mapleton, and Bensonhurst"))  
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "12"), lit("Boro Park, Kensington, Ocean Parkway, and Midwood"))   
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "13"), lit("Coney Island, Brighton Beach, Bensonhurst, Gravesend, and Seagate")) 
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "14"), lit("Flatbush, Midwood, Kensington, and Ocean Parkway")) 
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "15"), lit("Sheepshead Bay, Manhattan Beach, Kings Bay, Gerritsen Beach, Kings Highway, East Gravesend, Madison, Homecrest, and Plum Beach"))   
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "16"), lit("Brownsville and Ocean Hill"))  
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "17"), lit("East Flatbush, Remsen Village, Farragut, Rugby, Erasmus and Ditmas Village"))   
     .when((incendios.borough == "Brooklyn") & (incendios.community_District == "18"), lit("Canarsie, Bergen Beach, Mill Basin, Flatlands, Marine Park, Georgetown, and Mill Island"))   
    
     .when((incendios.borough == "Staten Island") & (incendios.community_District == "01"), lit("Arlington, Castleton Corners, Clifton, Concord, Elm Park, Fort Wadsworth, Graniteville, Grymes Hill, Livingston, Mariners Harbor, Meiers Corners, New Brighton, Port Ivory, Port Richmond, Randall Manor, Rosebank, St. George, Shore Acres, Silver Lake, Stapleton, Sunnyside, Tompkinsville, West Brighton, Westerleigh")) 
     .when((incendios.borough == "Staten Island") & (incendios.community_District == "02"), lit("Arrochar, Bloomfield, Bulls Heads, Chelsea, Dongan Hills, Egbertville, Emerson Hill, Grant City, Grasmere, High Rock, Lighthouse Hill, Midland Beach, New Dorp, New Springville, Oakwood, Ocean Breeze, Old Town, Richmondtown, South Beach, Todt Hill, and Travis")) 
     .when((incendios.borough == "Staten Island") & (incendios.community_District == "03"), lit("Annadale, Arden Heights, Bay Terrace, Charleston, Eltingville, Great Kills, Greenridge, Huguenot, Pleasant Plains, Prince's Bay, Richmondtown, Richmond Valley, Rossville, Tottenville, and Woodrow")) 
     
     .when((incendios.borough == "Queens") & (incendios.community_District == "07"), lit("Flushing, Bay Terrace, College Point, Whitestone, Malba, Beechhurst, Queensboro Hill, Willets Point, Auburndale, Clearview, Linden Hill, and Murray Hill")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "11"), lit("Bayside, Douglaston, Little Neck, Auburndale, East Flushing, Oakland Gardens, and Hollis Hills")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "01"), lit("Astoria, Old Astoria, Long Island City, Queensbridge, Ditmars, Ravenswood, Steinway, Garden Bay, and Woodside")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "09"), lit("Richmond Hill, Woodhaven, Ozone Park, and Kew Gardens")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "05"), lit("Ridgewood, Maspeth, Middle Village and Glendale")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "08"), lit("Fresh Meadows, Cunningham Heights, Hilltop Village, Pomonak Houses, Fresh Meadows, Jamaica Estates, Holliswood, Flushing South, Utopia, Kew Gardens Hills, and Briarwood")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "03"), lit("Jackson Heights, East Elmhurst, North Corona, and La Guardia Airport")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "02"), lit("Long Island City, Woodside, Sunnyside, and Maspeth")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "06"), lit("Forest Hills and Rego Park")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "10"), lit("Howard Beach, Ozone Park, South Ozone Park, Richmond Hill")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "12"), lit("Jamaica, Hollis, St. Albans, Springfield Gardens, Baisley Park, Rochdale Village, and South Jamaica")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "04"), lit("Corona, Corona Heights, Elmhurst, and Newtown")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "13"), lit("Queens Village, Glen Oaks, New Hyde Park, Bellerose, Cambria Heights, Laurelton, Rosedale, Floral Park, and Brookville")) 
     .when((incendios.borough == "Queens") & (incendios.community_District == "14"), lit("Breezy Point, Belle Harbor, Broad Channel, Neponsit, Arverne, Bayswater, Edgemere, Rockaway Park, Rockaway and Far Rockaway"))
     
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "01"), lit("Tribeca, Seaport/Civic Center, Financial District, Battery Park City"))   
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "02"), lit("Greenwich Village, West Village, NoHo, SoHo, Lower East Side, Chinatown, Little Italy")) 
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "03"), lit("Tompkins Square, East Village, Lower East Side, Chinatown, Two Bridges")) 
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "04"), lit("Clinton, Chelsea")) 
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "05"), lit("Midtown")) 
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "06"), lit("Stuyvesant Town, Tudor City, Turtle Bay, Peter Cooper Village, Murray Hill, Gramercy Park, Kips Bay, Sutton Place")) 
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "07"), lit("Manhattan Valley, Upper West Side, and Lincoln Square")) 
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "08"), lit("Upper East Side, LenoxHill, Yorkville, and Roosevelt Island")) 
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "09"), lit("Hamilton Heights, Manhattanville, Morningside Heights, and West Harlem")) 
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "10"), lit("Central Harlem"))   
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "11"), lit("East Harlem"))  
     .when((incendios.borough == "Manhattan") & (incendios.community_District == "12"), lit("Inwood and Washington Heights"))   
           
     .otherwise(lit("Sem Informação")) 
  )

In [21]:
incendios.show()

+----+
|year|
+----+
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
|2016|
+----+
only showing top 20 rows



In [17]:
incendios.printSchema()

root
 |-- year: integer (nullable = true)



In [24]:
#save to hive delta table
incendios \
    .select ("case_Year","borough","date","time","am_pm", "battalion", "community_District","precinct","incident_Classification","cause_Fire_Description","fire_Code_Category","Neighborhoods") \
    .write \
    .mode("overwrite") \
    .partitionBy("case_Year","borough") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/Incendios/")

In [25]:
spark.sql(
    """
    SELECT *
    FROM PL7G1_SILVER.Incendios
    """
).show()

+----------+--------+-----+---------+------------------+--------------------+--------+-----------------------+----------------------+------------------+---------+--------+
|      date|    time|am_pm|battalion|community_District|       neighborhoods|precinct|incident_Classification|cause_Fire_Description|fire_Code_Category|case_Year| borough|
+----------+--------+-----+---------+------------------+--------------------+--------+-----------------------+----------------------+------------------+---------+--------+
|2017-11-09|07:00:00|   AM|       42|                10|Bay Ridge, Dyker ...|      68|   MD Residential St...|     SIU Investigation|             Other|     2017|Brooklyn|
|2017-01-17|08:57:00|   AM|       43|                15|Sheepshead Bay, M...|      61|                  Other|     SIU Investigation|             Other|     2017|Brooklyn|
|2016-11-23|12:12:00|   PM|       42|                10|Bay Ridge, Dyker ...|      68|                    MFA|                   MFA|       

In [26]:
spark.sql(
    """
    SELECT Avg(case_Year)
    FROM  PL7G1_SILVER.Incendios
    WHERE battalion = "Sem Informação" 
    """
).show() 

+------------------+
|    avg(case_Year)|
+------------------+
|2018.5301948051947|
+------------------+



In [32]:
spark.sql(
    """
    SELECT *
    FROM  PL7G1_SILVER.Incendios
    WHERE precinct = "5" 
    """
).show()

+----------+--------+-----+--------------+------------------+--------------------+--------+-----------------------+----------------------+--------------------+---------+---------+
|      date|    time|am_pm|     battalion|community_District|       neighborhoods|precinct|incident_Classification|cause_Fire_Description|  fire_Code_Category|case_Year|  borough|
+----------+--------+-----+--------------+------------------+--------------------+--------+-----------------------+----------------------+--------------------+---------+---------+
|2017-04-27|08:00:00|   AM|Sem Informacao|                01|Flushing Avenue, ...|       5|                  Other|       Other - No Fire|             No Fire|     2017| Brooklyn|
|2016-12-31|03:46:00|   AM|Sem Informacao|                03|Tompkins Square, ...|       5|         Sem Informacao|  Preliminary Inves...|               Other|     2017|Manhattan|
|2017-01-02|03:46:29|   AM|             4|                03|Tompkins Square, ...|       5|   MD Res